In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export LD_LIBRARY_PATH=/Users/jamesmo/projects/sqa-utils/lib
import cx_Oracle

from sqa import query

try:
    cx_Oracle.init_oracle_client('/Users/jamesmo/projects/sqa-utils/lib')
except:
    pass

query.connect()


In [ ]:
scs = query.get_all_sc()


In [ ]:
import numpy as np
import pandas as pd

# Convert SC IDs to pandas
scs = np.array(scs)
scids = pd.DataFrame(scs, columns=['name', 'id'])

scids.query('name == "JWST"')


In [ ]:
# Check for duplicates
dups = scids.loc[list(set(scids.index) - set(scids.drop_duplicates(subset='id', keep=False).index))].sort_values('id')
dups
dups.sort_values('name')


In [ ]:

scid = int(df[df.name=='MMS1'].id)
scid = 170

_, tracks = query.get_tracks(scid)

# Convert tracks to dataframe
tracks = pd.DataFrame(tracks, columns=['track'])
tracks


In [ ]:
# Retrieve the monitor data
data = query.get_monitor_data(tracks.track, count=10)
sub  = list(data.keys())

tracks.query(f'track in {sub}')


In [ ]:
# Current working MMS monitor queries
qString = """
SELECT
    dcc.SCHEDULE_ITEM_ID,
    dcc.GROUND_ANTENNA_ID,
    dcc.RECEIVED_AT_TS,
    si.BEGINNING_OF_TRACK_TIME_DT,
    si.END_OF_TRACK_TIME_DT,
    dcc.TLM_GOOD_FRAME_COUNT,
    dcc.TLM_BAD_FRAME_COUNT,
    dcc.CAR_TRK_LOOP_LOCK_STATUS,
    dcc.CARRIER_SYSTEM_NOISE_TEMP,
    dcc.AGC_VOLTAGE
FROM
    SQADW.TF_DCC dcc
INNER JOIN
    SQADW.TL_SCHEDULE_ITEM si
ON  dcc.SCHEDULE_ITEM_ID = si.SCHEDULE_ITEM_ID
WHERE
    dcc.SPACECRAFT_ID = '{mission_id}'
ORDER BY
    dcc.RECEIVED_AT_TS
"""

missions = ['MMS1', 'MMS2', 'MMS3', 'MMS4']
for mission in missions:
    # Retrieve the data from SQA
    scid = int(scids.query(f'name == "{mission}"').id)
    desc, data = query.exec_query(qString.format(mission_id=scid))
    df = query.create_dict(desc, data, df=True)
    # Now save to h5
    ants = np.unique(df.GROUND_ANTENNA_ID)
    for ant in ants:
        nf = df[df.GROUND_ANTENNA_ID == ant]
        tracks = np.unique(nf.SCHEDULE_ITEM_ID)
        for track in tracks:
            tf = nf[nf.SCHEDULE_ITEM_ID == track]
            tf.to_hdf('/Users/jamesmo/projects/sqa-utils/local/data/tracks.h5', f'{mission}/{ant}/{track}')


In [ ]:
from datetime import datetime as dtt

def timestamp_to_datetime(timestamps):
    """
    Converts an integer or floating point timestamp to a Python datetime object.

    Parameters
    ----------
    timestamps: single or iterable of int or float
        A singular or a list of timestamps to convert

    Returns
    -------
    list of or single datetime
    """
    if isinstance(timestamps, (int, float)):
        return dtt.fromtimestamp(timestamps)
    else:
        return [dtt.fromtimestamp(ts) for ts in timestamps]


df.index = timestamp_to_datetime(df.index)

for name, column in df.items():
    if 'DT' in name:
        df[name] = timestamp_to_datetime(column)

def decode_strings(df):
    """
    Attempts to apply string.decode() to any column with a dtype of object.

    Parameters
    ----------
    df: pandas.DataFrame
        The DataFrame object to iterate over searching for object dtype columns
        to apply decoding to

    Returns
    -------
    df: pandas.DataFrame
        Same DataFrame object as input but with decoded columns
    """
    for name, column in df.items():
        if column.dtype == 'object':
            try:
                df[name] = column.apply(lambda string: string.decode())
                print(f'Decoded column {name}')
            except:
                print(f'Failed to decode column {name}')

    return df

df = decode_strings(df)


In [ ]:
# Pull down DR information and save to H5
from tqdm import tqdm

missions = ['MMS1', 'MMS2', 'MMS3', 'MMS4']
for mission in tqdm(missions, desc='Missions Processed'):
    scid = int(scids.query(f'name == "{mission}"').id)
    _, tracks = query.get_tracks(scid)
    drs = query.get_dr_data(tracks)
    drs = pd.concat(drs.values())
    drs.to_hdf('/Users/jamesmo/projects/sqa-utils/local/data/drs.h5', mission)
